In [31]:
# allows to import own functions
import sys, os, re
root_project = re.findall(r'(^\S*TFM_AGM)', os.getcwd())[0]
sys.path.append(root_project)

from src.utils.help_func import results_searchcv
from src.features.add_features import features_graph, features_pop

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import json
import seaborn as sns
sns.set()
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [33]:
df_1 = pd.read_pickle(f'{root_project}/data/processed/sir_simulation_50k_rev10.pickle')
df_2 = pd.read_pickle(f'{root_project}/data/processed/sir_simulation_50k_v2_rev10.pickle')
df_main = pd.concat([df_1, df_2])
df_main.reset_index(inplace=True, drop=True)
# add new features to the sir simulation results
df_main = features_graph(df_main)
df_main = features_pop(df_main)
# divide de data in test, validation and train sets
train_size = 0.6
test_val_size = (1 - train_size) / 2
test_val_size = int(df_main.shape[0] * test_val_size)

df_test = df_main.iloc[:test_val_size].sample(frac=1).reset_index(drop=True)
df_val = df_main.iloc[test_val_size:test_val_size*2].sample(frac=1).reset_index(drop=True)
df_train = df_main.iloc[test_val_size*2:].sample(frac=1).reset_index(drop=True)

df_test.to_pickle(f"{root_project}/data/processed/test_set.pickle")
df_val.to_pickle(f"{root_project}/data/processed/val_set.pickle")
df_train.to_pickle(f"{root_project}/data/processed/train_set.pickle")

In [35]:
df_test

,initial_country,idx_country,R0,Tc,Tr,omega,inf_power_1,inf_power_2,gradient_inf,mort_power_1,...,limit_deaths,n_closed,react_time,total_infected,total_death,total_recovered,betweenness,degree,closeness,country_pop
0,TGO,183,10.398646,0.310942,3.233376,0.023474,5.634880e+05,549.258242,"[2644.0, 6486.0, 25297.5, 98172.5, 373606.5, 1...",0.021317,...,403,14,12,7889088.0,185166.0,7889069.0,0.000387,0.117117,0.416510,7889094.0
1,GMB,68,7.360485,2.641248,19.440864,0.061777,1.613000e+03,51.011858,"[165.0, 192.5, 255.5, 339.0, 450.0, 597.0, 791...",0.008104,...,356,18,23,2279198.0,140693.0,2279143.0,0.000142,0.054054,0.375000,2280102.0
2,IMN,91,8.619569,1.399286,12.061244,0.054516,5.536786e+03,95.923875,"[362.0, 474.0, 763.5, 1221.0, 1928.5, 2984.0, ...",0.014178,...,828,2,16,84553.0,4561.0,84507.0,0.000000,0.018018,0.382759,84584.0
3,MHL,119,21.783851,0.562364,12.250448,0.041536,4.170429e+03,23.082604,"[1713.0, 3063.5, 7464.0, 15273.0, 17829.5, 563...",0.022505,...,164,5,19,58783.0,2397.0,58725.0,0.003000,0.036036,0.387435,58791.0
4,CHE,178,9.470525,1.038182,9.832126,0.104296,6.487757e+04,5306.208955,"[231.0, 331.0, 616.0, 1146.0, 2132.5, 3968.0, ...",0.011444,...,600,15,11,8516475.0,888165.0,8516428.0,0.008366,0.545045,0.552239,8516543.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,UGA,191,7.607418,0.683676,5.201008,0.029155,1.733401e+06,41266.587819,"[896.0, 1466.5, 3330.0, 7558.5, 17154.0, 38912...",0.004840,...,775,19,13,42723125.0,1245554.0,42723085.0,0.001707,0.117117,0.429400,42723139.0
19996,BGR,28,8.145659,1.019176,8.301857,0.073918,6.459729e+04,5118.131086,"[230.0, 329.0, 612.5, 1139.5, 2119.0, 3940.5, ...",0.009512,...,79,16,17,7024021.0,519142.0,7023991.0,0.000075,0.216216,0.456790,7024216.0
19997,MEX,122,10.662951,0.439558,4.686987,0.057469,9.013620e+06,257389.832714,"[554.0, 1126.0, 3448.5, 10557.5, 32317.0, 9889...",0.017385,...,673,12,1,126190778.0,7252034.0,126190739.0,0.005093,0.225225,0.484716,126190788.0
19998,SWZ,60,2.470674,4.721145,11.664411,0.155050,4.107143e+01,4.263736,"[11.0, 12.0, 14.0, 15.5, 17.0, 19.5, 22.5, 25....",0.052174,...,563,4,26,1015884.0,157406.0,1015877.0,0.000000,0.009009,0.347418,1136191.0


In [36]:
df_val

,initial_country,idx_country,R0,Tc,Tr,omega,inf_power_1,inf_power_2,gradient_inf,mort_power_1,...,limit_deaths,n_closed,react_time,total_infected,total_death,total_recovered,betweenness,degree,closeness,country_pop
0,KIR,99,15.853136,1.201922,19.054229,0.037137,8.199500e+03,82.293413,"[774.0, 1067.5, 1866.0, 3215.5, 5408.0, 8715.0...",0.008851,...,632,9,27,115829.0,4229.0,115747.0,0.000956,0.036036,0.391534,115847.0
1,GHA,71,18.066542,1.212901,21.912933,0.289685,2.391207e+04,3143.146067,"[139.0, 193.0, 343.0, 610.5, 1086.0, 1932.0, 3...",0.016080,...,769,18,1,29767083.0,8622929.0,29766918.0,0.003208,0.198198,0.483660,29767108.0
2,LKA,171,15.962824,0.513399,8.195293,0.280759,1.547852e+06,82069.805556,"[329.0, 629.0, 1778.0, 5023.5, 14189.0, 40045....",0.062761,...,662,9,19,21669991.0,6083980.0,21669925.0,0.000759,0.207207,0.462500,21670000.0
3,TLS,182,5.705309,3.791162,21.629752,0.023398,1.234286e+03,14.479464,"[259.0, 287.5, 350.0, 425.5, 517.5, 629.5, 765...",0.003819,...,935,12,18,1264585.0,29472.0,1264534.0,0.000000,0.027027,0.359223,1267972.0
4,OMN,142,10.390069,0.663658,6.895452,0.180061,3.408063e+05,19378.184971,"[236.0, 397.0, 938.0, 2214.5, 5225.5, 12313.5,...",0.034645,...,679,12,15,4829472.0,869548.0,4829428.0,0.000685,0.225225,0.458678,4829483.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,TKM,188,10.633958,0.377346,4.012678,0.047041,4.179111e+05,2039.517621,"[1093.0, 2403.0, 8164.0, 27686.0, 93244.5, 306...",0.037120,...,490,5,15,5850902.0,275205.0,5850873.0,0.000014,0.081081,0.425287,5850908.0
19996,BIH,23,13.502059,0.474031,6.400394,0.036139,2.374057e+05,1721.176471,"[1296.0, 2560.0, 7549.0, 22197.0, 64703.5, 183...",0.021425,...,624,10,19,3323921.0,120082.0,3323878.0,0.000007,0.090090,0.403636,3323929.0
19997,USA,195,20.943345,0.547257,11.461387,0.181424,1.220716e+07,680305.439716,"[736.0, 1377.0, 3773.0, 10338.5, 28326.5, 7760...",0.010332,...,963,16,21,327167424.0,59356066.0,327167325.0,0.184702,0.819820,0.620112,327167434.0
19998,BGR,28,2.469223,4.801343,11.855587,0.211835,2.900000e+01,4.134328,"[8.0, 8.5, 10.0, 11.5, 12.5, 14.0, 15.5, 17.5,...",0.071429,...,775,14,1,6279121.0,1330011.0,6279121.0,0.000075,0.216216,0.456790,7024216.0


In [37]:
df_train

,initial_country,idx_country,R0,Tc,Tr,omega,inf_power_1,inf_power_2,gradient_inf,mort_power_1,...,limit_deaths,n_closed,react_time,total_infected,total_death,total_recovered,betweenness,degree,closeness,country_pop
0,GRD,75,19.870765,0.480735,9.552574,0.205120,7954.642857,185.218884,"[459.0, 908.5, 2663.0, 7573.0, 19662.0, 38508....",0.115961,...,664,2,13,111446.0,22816.0,111396.0,0.001091,0.063063,0.392226,111454.0
1,JOR,96,11.224467,1.673681,18.786173,0.042672,22736.214286,422.626794,"[569.0, 724.0, 1118.0, 1726.0, 2664.0, 4111.5,...",0.003830,...,213,8,29,9955930.0,424722.0,9955836.0,0.002139,0.306306,0.497758,9956011.0
2,TUR,187,21.418414,0.913081,19.556749,0.247430,243162.000000,21323.148026,"[318.0, 484.5, 990.0, 2023.0, 4135.5, 8453.0, ...",0.011711,...,874,8,24,82319709.0,20368223.0,82319545.0,0.043460,0.833333,0.606557,82319724.0
3,SWZ,60,10.751916,0.272199,2.926663,0.070168,81149.857143,91.400568,"[1173.0, 3123.5, 13453.5, 56844.0, 220481.0, 4...",0.067114,...,601,3,29,1136185.0,79703.0,1136168.0,0.000000,0.009009,0.347418,1136191.0
4,IDN,87,6.422787,3.015805,19.369874,0.091391,591.714286,30.684932,"[81.0, 93.0, 119.5, 152.5, 195.5, 250.5, 320.0...",0.013279,...,781,11,17,267375242.0,24435597.0,267375194.0,0.003337,0.180180,0.418868,267663435.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,DOM,53,6.017172,1.104508,6.646015,0.082099,19916.071429,2558.611465,"[120.0, 164.5, 288.0, 505.5, 887.0, 1556.0, 27...",0.013751,...,783,3,16,10620734.0,871908.0,10620720.0,0.003726,0.238739,0.478448,10627165.0
59996,TON,184,19.218150,0.763432,14.671755,0.090251,7349.500000,91.590551,"[784.0, 1260.0, 2761.0, 5879.5, 11752.5, 20423...",0.035066,...,118,18,5,103189.0,9248.0,103114.0,0.000000,0.027027,0.332335,103197.0
59997,MDA,124,14.041589,1.079631,15.159733,0.020774,197134.285714,1623.308096,"[1476.0, 2110.0, 3921.5, 7279.5, 13487.0, 2490...",0.002350,...,575,3,25,3545860.0,73581.0,3545764.0,0.000025,0.135135,0.438735,3545883.0
59998,GRD,75,11.299175,0.777449,8.784536,0.144687,7952.357143,228.078947,"[266.0, 421.5, 910.0, 1944.0, 4074.0, 8189.0, ...",0.068174,...,605,14,15,111446.0,16079.0,111398.0,0.001091,0.063063,0.392226,111454.0
